In [ ]:
pip install roboflow supervision opencv-python-headless matplotlib pillow

In [3]:
# Import library yang akan digunakan
from roboflow import Roboflow
import cv2
import matplotlib.pyplot as plt
from google.colab import files
import numpy as np

In [4]:
# Inisialisasi API
API_KEY = "k99zb7V6Xl2r89bKBGid"
PROJECT_NAME = "sirkelabang"
MODEL_VERSION = 1

In [5]:
rf = Roboflow(api_key=API_KEY)
project = rf.workspace().project(PROJECT_NAME)
model = project.version(MODEL_VERSION).model

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
def upload_and_predict():
    """
    Upload gambar dan lakukan prediksi menggunakan model Roboflow.
    """
    # Upload gambar
    print("Silakan upload gambar:")
    uploaded = files.upload()
    image_path = list(uploaded.keys())[0]

    # Baca gambar dan prediksi
    image = cv2.imread(image_path)
    print("Memproses gambar...")
    result = model.predict(image_path, confidence=50).json()

    # Gambar hasil prediksi
    for pred in result['predictions']:
        # Hitung koordinat box
        x1 = int(pred['x'] - pred['width'] // 2)
        y1 = int(pred['y'] - pred['height'] // 2)
        x2 = int(pred['x'] + pred['width'] // 2)
        y2 = int(pred['y'] + pred['height'] // 2)

        # Gambar mask jika ada
        if 'mask' in pred:
            mask = np.array(pred['mask'], dtype=np.uint8)
            mask = cv2.resize(mask, (int(pred['width']), int(pred['height'])))
            mask_area = np.where(mask > 0)
            image[y1 + mask_area[0], x1 + mask_area[1]] = (0, 255, 255)

        # Gambar box dan label
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        label = f"{pred['class']} ({pred['confidence'] * 100:.1f}%)"
        cv2.putText(image, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Tampilkan hasil
    plt.figure(figsize=(12, 12))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.title("Hasil Prediksi")
    plt.show()

# Jalankan fungsi
upload_and_predict()